# Imports

In [58]:
import os
from pathlib import Path

import pandas as pd
from deepface import DeepFace
from tqdm.autonotebook import tqdm

# Initiating Database

In [9]:
faces_found = DeepFace.find(
    # img_path=os.path.join(
    #     "data", "database", "incoming_users", "authorized_users", "1", "000023.jpg"
    # ),
    # img_path=os.path.join(
    #     "data", "database", "incoming_users", "unauthorized_users", "101", "020633.jpg"
    # ),
    img_path=os.path.join(
        "data", "database", "incoming_users", "authorized_users", "22", "001677.jpg"
    ),
    db_path=os.path.join("data", "database", "authorized_users"),
    enforce_detection=False,
)

24-04-03 15:34:04 - Searching data\database\incoming_users\authorized_users\22\001677.jpg in 1826 length datastore
24-04-03 15:34:05 - find function duration 1.2937004566192627 seconds


In [10]:
faces_found[0]

,identity,hash,target_x,target_y,target_w,target_h,source_x,source_y,source_w,source_h,threshold,distance
0,data\database\authorized_users\63\148723.jpg,7784b838f6009d7c18db658c2c078f3220cc21a1,37,74,101,101,0,0,178,218,0.68,0.641510
1,data\database\authorized_users\55\147118.jpg,752859037bbb74e5afcf1be196bec0bad056ba82,32,69,113,113,0,0,178,218,0.68,0.653444
2,data\database\authorized_users\15\040569.jpg,b96cca437e50c2509b3c7d688df40015d081e774,0,0,178,218,0,0,178,218,0.68,0.672288


In [11]:
faces_found[0]["identity"].apply(lambda x: Path(x).parts[3]).unique()

array(['63', '55', '15'], dtype=object)

# Testing Authorized Users

In [12]:
testing_authorized_users = pd.DataFrame(
    columns=["image_path", "truth_label", "predicted_label", "cosine_distance"]
)

In [13]:
testing_authorized_users

,image_path,truth_label,predicted_label,cosine_distance


In [16]:
incoming_authorized_users_path = os.path.join(
    "data", "database", "incoming_users", "authorized_users"
)

for person_id in tqdm(
    iterable=os.listdir(incoming_authorized_users_path), desc="Processing users"
):
    for person_photo in tqdm(
        iterable=os.listdir(os.path.join(incoming_authorized_users_path, person_id)),
        desc="Processing user photos",
        leave=False,
    ):
        faces_found = DeepFace.find(
            img_path=os.path.join(
                incoming_authorized_users_path, person_id, person_photo
            ),
            db_path=os.path.join("data", "database", "authorized_users"),
            enforce_detection=False,
            silent=True,
        )

        predicted_user_name = faces_found[0]["identity"].apply(
            lambda x: Path(x).parts[3]
        )[
            0
        ]  # get best match

        predicted_cosine_distance = faces_found[0]["distance"]

        df_tested_user = pd.DataFrame(
            {
                "image_path": os.path.join(
                    incoming_authorized_users_path, person_id, person_photo
                ),
                "truth_label": person_id,
                "predicted_label": predicted_user_name,
                "cosine_distance": predicted_cosine_distance,
            },
            index=[0],
        )

        testing_authorized_users = pd.concat(
            [testing_authorized_users, df_tested_user], ignore_index=True
        )

Processing users: 100%|██████████| 50/50 [03:43<00:00,  4.46s/it]    


In [59]:
acceptance_threshold = 0.5

testing_authorized_users["grant_access"] = (
    testing_authorized_users["cosine_distance"] < acceptance_threshold
) * 1

In [60]:
testing_authorized_users["is_match"] = (
    testing_authorized_users["truth_label"]
    == testing_authorized_users["predicted_label"]
) * 1

In [61]:
testing_authorized_users["is_match"].sum() / len(testing_authorized_users)

0.9146919431279621

In [62]:
testing_authorized_users["grant_access"].sum() / len(testing_authorized_users)

0.8767772511848341

In [63]:
testing_authorized_users

,image_path,truth_label,predicted_label,cosine_distance,grant_access,is_match
0,data\database\incoming_users\authorized_users\...,50,50,0.351713,1,1
1,data\database\incoming_users\authorized_users\...,50,50,0.366908,1,1
2,data\database\incoming_users\authorized_users\...,50,50,0.369449,1,1
3,data\database\incoming_users\authorized_users\...,1,1,0.330864,1,1
4,data\database\incoming_users\authorized_users\...,1,8,0.579042,0,0
...,...,...,...,...,...,...
206,data\database\incoming_users\authorized_users\...,49,49,0.426673,1,1
207,data\database\incoming_users\authorized_users\...,49,49,0.454959,1,1
208,data\database\incoming_users\authorized_users\...,49,49,0.338384,1,1
209,data\database\incoming_users\authorized_users\...,49,49,0.338594,1,1


# Testing Unauthorized Users

In [21]:
testing_unauthorized_users = pd.DataFrame(
    columns=["image_path", "truth_label", "predicted_label", "cosine_distance"]
)

In [22]:
incoming_unauthorized_users_path = os.path.join(
    "data", "database", "incoming_users", "unauthorized_users"
)

for person_id in tqdm(
    iterable=os.listdir(incoming_unauthorized_users_path), desc="Processing users"
):
    for person_photo in tqdm(
        iterable=os.listdir(os.path.join(incoming_unauthorized_users_path, person_id)),
        desc="Processing user photos",
        leave=False,
    ):
        faces_found = DeepFace.find(
            img_path=os.path.join(
                incoming_unauthorized_users_path, person_id, person_photo
            ),
            db_path=os.path.join("data", "database", "authorized_users"),
            enforce_detection=False,
            silent=True,
        )

        if not faces_found[0].empty:
            predicted_user_name = faces_found[0]["identity"].apply(
                lambda x: Path(x).parts[3]
            )[
                0
            ]  # get best match
            predicted_cosine_distance = faces_found[0]["distance"]
        else:
            predicted_user_name = -1
            predicted_cosine_distance = -1

        df_tested_user = pd.DataFrame(
            {
                "image_path": os.path.join(
                    incoming_unauthorized_users_path, person_id, person_photo
                ),
                "truth_label": person_id,
                "predicted_label": predicted_user_name,
                "cosine_distance": predicted_cosine_distance,
            },
            index=[0],
        )

        testing_unauthorized_users = pd.concat(
            [testing_unauthorized_users, df_tested_user], ignore_index=True
        )

Processing users: 100%|██████████| 50/50 [04:03<00:00,  4.87s/it]    


In [64]:
acceptance_threshold = 0.5

testing_unauthorized_users["grant_access"] = (
    testing_unauthorized_users["cosine_distance"] < acceptance_threshold
) * 1

In [65]:
testing_unauthorized_users["is_match"] = (
    testing_unauthorized_users["truth_label"]
    == testing_unauthorized_users["predicted_label"]
) * 1

In [66]:
testing_unauthorized_users["is_match"].sum() / len(testing_unauthorized_users)

0.0

In [67]:
testing_unauthorized_users["grant_access"].sum() / len(testing_unauthorized_users)

0.14018691588785046

In [68]:
testing_unauthorized_users

,image_path,truth_label,predicted_label,cosine_distance,is_match,grant_access
0,data\database\incoming_users\unauthorized_user...,150,43,0.558697,0,0
1,data\database\incoming_users\unauthorized_user...,150,27,0.528804,0,0
2,data\database\incoming_users\unauthorized_user...,150,89,0.608577,0,0
3,data\database\incoming_users\unauthorized_user...,101,-1,-1.000000,0,1
4,data\database\incoming_users\unauthorized_user...,101,78,0.605179,0,0
...,...,...,...,...,...,...
209,data\database\incoming_users\unauthorized_user...,148,-1,-1.000000,0,1
210,data\database\incoming_users\unauthorized_user...,148,41,0.603607,0,0
211,data\database\incoming_users\unauthorized_user...,148,41,0.642242,0,0
212,data\database\incoming_users\unauthorized_user...,148,41,0.661102,0,0


In [69]:
testing_unauthorized_users.predicted_label.value_counts()

predicted_label
23     17
-1     11
27      8
89      8
97      8
       ..
63      1
80      1
50      1
72      1
100     1
Name: count, Length: 68, dtype: int64